In [1]:
import turicreate as tc
import numpy as np

In [2]:
data = tc.SFrame('home_data/')

In [3]:
data.head()

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650.0,1.0,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242.0,2.0,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000.0,1.0,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000.0,1.0,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080.0,1.0,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930.0,1.0,0
1321400060,2014-06-27 00:00:00+00:00,257500.0,3.0,2.25,1715.0,6819.0,2.0,0
2008000270,2015-01-15 00:00:00+00:00,291850.0,3.0,1.5,1060.0,9711.0,1.0,0
2414600126,2015-04-15 00:00:00+00:00,229500.0,3.0,1.0,1780.0,7470.0,1.0,0
3793500160,2015-03-12 00:00:00+00:00,323000.0,3.0,2.5,1890.0,6560.0,2.0,0


In [4]:
def get_numpy_data(data_frame, features, output):
    data_frame['constant'] = 1
    features = ['constant'] + features
    features_sframe = data_frame[features]
    features_matrix = features_sframe.to_numpy()
    output_sarray = data_frame[output]
    output_array = output_sarray.to_numpy()
    print(features_matrix)
    return features_matrix, output_array

In [5]:
def weight(features_matrix, output_array):
    features_matrix_transpose = np.transpose(features_matrix)
    XTX = np.linalg.inv(np.dot(features_matrix_transpose, features_matrix))
    W = np.dot(np.dot(XTX, features_matrix_transpose), output_array)
    return W

In [6]:
def predict_outcome(feature_matrix, weights):
    predictions = np.dot(feature_matrix, weights)
    return predictions

In [7]:
def error(predictions, output_array):
    return predictions - output_array

In [8]:
def feature_derivative(errors, feature):
    derivative = 2 * np.dot(feature, errors)
    return derivative

In [9]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False
    weights = np.array(initial_weights)
    gradient_magnitude = 0
    while not converged:
        predictions = predict_outcome(feature_matrix, weights)
        errors = predictions - output
        # compute the predictions based on feature_matrix and weights:
        # compute the errors as predictions - output:
    
        gradient_sum_squares = 0 # initialize the gradient
        # while not converged, update each weight individually:
        for i in range(len(weights)):
            # Recall that feature_matrix[:, i] is the feature column associated with weights[i]
            # compute the derivative for weight[i]:
            derivate = feature_derivative(errors, feature_matrix[:, i])
            # add the squared derivative to the gradient magnitude
            gradient_sum_squares += derivate**2
            # update the weight based on step size and derivative:
            weights[i] = weights[i] - step_size * derivate
        gradient_magnitude = np.sqrt(gradient_sum_squares)
        if gradient_magnitude < tolerance:
            converged = True
    return(weights)

In [10]:
train_data,test_data = data.random_split(.8,seed=0)

In [11]:
simple_features = ['sqft_living']
my_output= 'price'
simple_feature_matrix, output = get_numpy_data(train_data, simple_features, my_output)
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

[[1.00e+00 1.18e+03]
 [1.00e+00 2.57e+03]
 [1.00e+00 7.70e+02]
 ...
 [1.00e+00 1.53e+03]
 [1.00e+00 1.60e+03]
 [1.00e+00 1.02e+03]]


In [12]:
simple_weights = regression_gradient_descent(simple_feature_matrix, output, initial_weights, step_size, tolerance)
print(simple_weights)

[-46999.88716555    281.91211912]


## 9. Quiz Question: What is the value of the weight for sqft_living -- the second element of ‘simple_weights’ (rounded to 1 decimal place)?
### 281.9

In [13]:
test_simple_feature_matrix, test_output = get_numpy_data(test_data, simple_features, my_output)

[[1.00e+00 1.43e+03]
 [1.00e+00 2.95e+03]
 [1.00e+00 1.71e+03]
 ...
 [1.00e+00 2.52e+03]
 [1.00e+00 2.31e+03]
 [1.00e+00 1.02e+03]]


In [14]:
predictions = predict_outcome(test_simple_feature_matrix, simple_weights)
predictions[0]

356134.4431709297

## 11. Quiz Question: What is the predicted price for the 1st house in the Test data set for model 1 (round to nearest dollar)?
### - $356134

In [15]:
def rss(real, predicted):
    return sum((real - predicted)**2)

In [16]:
rss(test_output, predictions)

275400047593155.7

In [17]:
model_features = ['sqft_living', 'sqft_living15']
my_output = 'price'
feature_matrix, output = get_numpy_data(train_data, model_features,my_output)
initial_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9 

[[1.00e+00 1.18e+03 1.34e+03]
 [1.00e+00 2.57e+03 1.69e+03]
 [1.00e+00 7.70e+02 2.72e+03]
 ...
 [1.00e+00 1.53e+03 1.53e+03]
 [1.00e+00 1.60e+03 1.41e+03]
 [1.00e+00 1.02e+03 1.02e+03]]


In [18]:
multi_weights = regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance)
print(multi_weights)

[-9.99999688e+04  2.45072603e+02  6.52795277e+01]


In [19]:
test_simple_feature_matrix, test_output = get_numpy_data(test_data, model_features, my_output)

[[1.00e+00 1.43e+03 1.78e+03]
 [1.00e+00 2.95e+03 2.14e+03]
 [1.00e+00 1.71e+03 1.03e+03]
 ...
 [1.00e+00 2.52e+03 2.52e+03]
 [1.00e+00 2.31e+03 1.83e+03]
 [1.00e+00 1.02e+03 1.02e+03]]


In [20]:
predictions = predict_outcome(test_simple_feature_matrix, multi_weights)
predictions[0]

366651.4120365591

## 15. Quiz Question: What is the predicted price for the 1st house in the TEST data set for model 2 (round to nearest dollar)?
### - $366651

In [21]:
test_data['price'][0]

310000.0

## 17. Quiz Question: Which estimate was closer to the true price for the 1st house on the TEST data set, model 1 or model 2?
### - model 1

In [22]:
rss(test_output, predictions)

270263446465243.97

## 19. Quiz Question: Which model (1 or 2) has lowest RSS on all of the TEST data?  
### - model 2